In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'
if not os.path.exists(volume_dir):
    os.makedirs(volume_dir)

In [ ]:
##### reconstruct annotation volume ######

In [ ]:
downsample_factor = 8

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

stack = 'MD589'

n_sec = {'MD589': 445, 'MD594': 432}
    
# build annotation volume
section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end

In [ ]:
dm = DataManager(stack=stack)

volume = np.zeros((dm.image_height/downsample_factor, 
                   dm.image_width/downsample_factor, 
                   int(n_sec[stack]*z_xy_ratio_downsampled)), np.int8)

print volume.shape

for sec in range(section_bs_begin, section_bs_end+1):
    
    # only 9 class
    
#     fn = '/oasis/projects/nsf/csd395/yuncong/CSHL_labelMapsLossless/%(stack)s/%(stack)s_%(slice)04d_labelMapLossless.npz' % \
#         {'stack': stack, 'slice': sec}
    fn = '/oasis/projects/nsf/csd395/yuncong/CSHL_labelMapsLossless/%(stack)s/%(stack)s_%(slice)04d_labelMapLossless.bp' % \
        {'stack': stack, 'slice': sec}

    # or, all classes
#     fn = '/oasis/projects/nsf/csd395/yuncong/CSHL_labelMapsLossless/%(stack)s/%(stack)s_%(slice)04d_labelMapLossless_AllClasses.npz' % \
#         {'stack': stack, 'slice': sec}
        
    if os.path.exists(fn):
        print sec
#         label_map = np.load(fn)['arr_0'][::downsample_factor, ::downsample_factor]
        label_map = bp.unpack_ndarray_file(fn)[::downsample_factor, ::downsample_factor]
        volume[:, :, int(z_xy_ratio_downsampled*sec):int(z_xy_ratio_downsampled*(sec+1))] = label_map[..., np.newaxis]

    # np.savez_compressed(os.path.join(volume_dir, 'volume_'+stack+'_labelmap'), volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1])
    # np.savez_compressed(os.path.join(volume_dir, 'volume_'+stack+'_predMap'), volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1])
#     np.savez_compressed(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)d' % {'stack': stack, 'label': l}),
#                         volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1])

In [ ]:
# for sec in range(section_bs_begin, section_bs_end+1):

sec = 351
plt.matshow(volume[..., int(sec*z_xy_ratio_downsampled)]);
plt.title('section %d, z = (%d, %d)'%(sec, int(sec*z_xy_ratio_downsampled), 
                                     int(z_xy_ratio_downsampled*(sec+1))-1));
plt.show();

In [ ]:
ys, xs, zs = np.where(volume != 0)

xmin = xs.min()
xmax = xs.max()
ymin = ys.min()
ymax = ys.max()
zmin = zs.min()
zmax = zs.max()

print xmin, xmax
print ymin, ymax
print zmin, zmax

reduced_xdim = xmax+1-xmin
reduced_ydim = ymax+1-ymin
reduced_zdim = zmax+1-zmin
print reduced_xdim, reduced_ydim, reduced_zdim

In [ ]:
bp.pack_ndarray_file(volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1],
                     os.path.join(volume_dir, 'volume_%(stack)s_annotation.bp' % {'stack': stack}))

In [ ]:
np.savez_compressed(os.path.join(volume_dir, 
                                 'volume_%(stack)s_annotationAllClasses' % {'stack': stack}),
                    volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1])

In [ ]:
np.savez_compressed(os.path.join(volume_dir, 
                                 'volume_%(stack)s_annotation' % {'stack': stack}),
                    volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1])